In [ ]:
!nvidia-smi

Sun Jan 28 15:36:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install --quiet transformers
!pip install --quiet pytorch-lightning==1.1.1
!pip install --quiet tokenizers
!pip install --quiet sentencepiece
!pip install termcolor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.7/669.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.6 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 39.6 MB/s eta 0:00:00
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.1.1
    Uninstalling pytorch-lightning-1.1.1:
      Successfully uninstalled pytorch-lightning-1.1.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import argparse
import glob
import os

import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap
from keras.callbacks import ModelCheckpoint


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [ ]:
pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Seed set to 42


42

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import load_dataset

squad_v2 = False  # Change this to True if you want to load SQuAD v2

# Determine dataset name based on squad_v2 variable
dataset_name = "squad_v2" if squad_v2 else "squad"

# Load the dataset
datasets = load_dataset(dataset_name)
# Extract the first 100 data points
first_100_data = datasets["train"][:800]

# Convert the data to a DataFrame
df = pd.DataFrame(first_100_data)

# Filter out duplicate contexts
df_unique_contexts = df.drop_duplicates(subset='context')

# If there are fewer than 100 unique contexts, select the first 100 rows
if len(df_unique_contexts) < 100:
    df_unique_contexts = df_unique_contexts[:100]

# Display the DataFrame with unique contexts
df_unique_contexts

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
5,5733bf84d058e614000b61be,University_of_Notre_Dame,"As at most other universities, Notre Dame's st...",When did the Scholastic Magazine of Notre dame...,"{'text': ['September 1876'], 'answer_start': [..."
10,5733bed24776f41900661188,University_of_Notre_Dame,The university is the major seat of the Congre...,Where is the headquarters of the Congregation ...,"{'text': ['Rome'], 'answer_start': [119]}"
15,5733a6424776f41900660f51,University_of_Notre_Dame,The College of Engineering was established in ...,How many BS level degrees are offered in the C...,"{'text': ['eight'], 'answer_start': [487]}"
20,5733a70c4776f41900660f64,University_of_Notre_Dame,All of Notre Dame's undergraduate students are...,What entity provides help with the management ...,"{'text': ['Learning Resource Center'], 'answer..."
...,...,...,...,...,...
747,56bec1c53aeaaa14008c936b,Beyoncé,Beyoncé has stated that she is personally insp...,Beyonce has noted which first lady with saying...,"{'text': ['Michelle Obama'], 'answer_start': [..."
759,56bec29b3aeaaa14008c937f,Beyoncé,"In 2006, Beyoncé introduced her all-female tou...",Beyonce had an all-female tour band whose name...,"{'text': ['Suga Mama'], 'answer_start': [53]}"
771,56bec3303aeaaa14008c9391,Beyoncé,Beyoncé has received praise for her stage pres...,What characteristics has Beyonce received accl...,"{'text': ['stage presence and voice'], 'answer..."
779,56bec3ea3aeaaa14008c939f,Beyoncé,"Described as being ""sexy, seductive and provoc...",Beyonce self proclaimed alter ego is named what?,"{'text': ['Sasha Fierce'], 'answer_start': [139]}"


In [ ]:
# Extracting 'text' and 'answer_start' into separate arrays
text_array = [item['text'][0] for item in df_unique_contexts['answers']]
answer_start_array = [item['answer_start'][0] for item in df_unique_contexts['answers']]
contexts = df_unique_contexts['context'].values
questions = df_unique_contexts['question'].values
# Creating a new DataFrame with extracted arrays
new_data = {
    'question': questions,
    'context': contexts,
    'answer_text': text_array,
    'answer_start': answer_start_array
}

df_extracted = pd.DataFrame(new_data)

# Displaying the new DataFrame
df_extracted

,question,context,answer_text,answer_start
0,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",Saint Bernadette Soubirous,515
1,When did the Scholastic Magazine of Notre dame...,"As at most other universities, Notre Dame's st...",September 1876,248
2,Where is the headquarters of the Congregation ...,The university is the major seat of the Congre...,Rome,119
3,How many BS level degrees are offered in the C...,The College of Engineering was established in ...,eight,487
4,What entity provides help with the management ...,All of Notre Dame's undergraduate students are...,Learning Resource Center,496
...,...,...,...,...
95,Beyonce has noted which first lady with saying...,Beyoncé has stated that she is personally insp...,Michelle Obama,68
96,Beyonce had an all-female tour band whose name...,"In 2006, Beyoncé introduced her all-female tou...",Suga Mama,53
97,What characteristics has Beyonce received accl...,Beyoncé has received praise for her stage pres...,stage presence and voice,36
98,Beyonce self proclaimed alter ego is named what?,"Described as being ""sexy, seductive and provoc...",Sasha Fierce,139


In [ ]:
df = df.drop_duplicates(subset = ["question"]).reset_index(drop = True)

In [ ]:
# Path to the pre-trained T5 model if you are using Google Colab
MODEL_NAME = "t5-base"

In [ ]:
# Initialize a T5 tokenizer using a pre-trained model
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
class BioQADataset(Dataset):
    """
    Dataset class for BioASQ data to be used with T5 model.

    Parameters:
        data (pd.DataFrame): DataFrame containing the data.
        tokenizer (T5Tokenizer): Tokenizer for encoding text data.
        source_max_token_len (int): Maximum length for source encoding.
        target_max_token_len (int): Maximum length for target encoding.
    """

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: T5Tokenizer,
        source_max_token_len: int = 396,
        target_max_token_len: int = 32
    ):
        """
        Initializes the BioQADataset.

        Args:
            data (pd.DataFrame): DataFrame containing the data.
            tokenizer (T5Tokenizer): Tokenizer for encoding text data.
            source_max_token_len (int, optional): Maximum length for source encoding. Defaults to 396.
            target_max_token_len (int, optional): Maximum length for target encoding. Defaults to 32.
        """
        self.tokenizer = tokenizer
        self.data = data
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def __len__(self):
        """
        Returns the number of samples in the dataset.
        """
        return len(self.data)

    def __getitem__(self, index: int):
        """
        Gets a sample from the dataset at the specified index.

        Args:
            index (int): Index of the sample to retrieve.

        Returns:
            dict: A dictionary containing the encoded inputs and labels.
        """
        data_row = self.data.iloc[index]

        # Encode source (question and context)
        source_encoding = self.tokenizer(
            data_row["question"],
            data_row["context"],
            max_length=self.source_max_token_len,
            padding="max_length",
            truncation="only_second",
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        # Encode target (answer_text)
        target_encoding = self.tokenizer(
            data_row["answer_text"],
            max_length=self.target_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        # Prepare labels (setting padding tokens to -100)
        labels = target_encoding["input_ids"]
        labels[labels == 0] = -100

        return {
            "question": data_row["question"],
            "context": data_row["context"],
            "answer_text": data_row["answer_text"],
            "input_ids": source_encoding["input_ids"].flatten(),
            "attention_mask": source_encoding["attention_mask"].flatten(),
            "labels": labels.flatten()
        }

In [ ]:
# Create a sample dataset using BioQADataset class
sample_dataset = BioQADataset(df, tokenizer)

In [ ]:
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.05)

In [ ]:
class BioQADataModule(pl.LightningDataModule):
    """
    LightningDataModule for preparing data for training and testing a T5 model on BioASQ data.

    Parameters:
        train_df (pd.DataFrame): DataFrame containing training data.
        test_df (pd.DataFrame): DataFrame containing test/validation data.
        tokenizer (T5Tokenizer): Tokenizer for encoding text data.
        batch_size (int): Batch size for data loaders. Defaults to 8.
        source_max_token_len (int): Maximum length for source encoding. Defaults to 396.
        target_max_token_len (int): Maximum length for target encoding. Defaults to 32.
    """

    def __init__(
        self,
        train_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: T5Tokenizer,
        batch_size: int = 8,
        source_max_token_len: int = 396,
        target_max_token_len: int = 32
    ):
        """
        Initializes the BioQADataModule.

        Args:
            train_df (pd.DataFrame): DataFrame containing training data.
            test_df (pd.DataFrame): DataFrame containing test/validation data.
            tokenizer (T5Tokenizer): Tokenizer for encoding text data.
            batch_size (int, optional): Batch size for data loaders. Defaults to 8.
            source_max_token_len (int, optional): Maximum length for source encoding. Defaults to 396.
            target_max_token_len (int, optional): Maximum length for target encoding. Defaults to 32.
        """
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def setup(self, stage=None):
        """
        Setup datasets for training and testing.

        Args:
            stage (str): Stage of training, either 'fit' or 'test'. Defaults to None.
        """
        self.train_dataset = BioQADataset(
            self.train_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len
        )

        self.test_dataset = BioQADataset(
            self.train_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len
        )

    def train_dataloader(self):
        """
        Returns a DataLoader for the training dataset.
        """
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4
        )

    def val_dataloader(self):
        """
        Returns a DataLoader for the validation dataset.
        """
        return DataLoader(
            self.test_dataset,
            batch_size=1,
            num_workers=4
        )

    def test_dataloader(self):
        """
        Returns a DataLoader for the test dataset.
        """
        return DataLoader(
            self.test_dataset,
            batch_size=1,
            num_workers=4
        )

In [ ]:
# Set batch size and number of epochs
BATCH_SIZE = 8
N_EPOCHS = 100
# Create an instance of BioQADataModule
data_module = BioQADataModule(train_df, val_df, tokenizer, batch_size = BATCH_SIZE)

# Setup datasets for training and validation
data_module.setup()

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict = True)

In [ ]:
class BioQAModel(pl.LightningModule):
    """
    LightningModule for fine-tuning a T5 model on BioASQ data.

    Attributes:
        model (T5ForConditionalGeneration): Pre-trained T5 model.
        validation_step_outputs (list): List to store validation step outputs.
    """

    def __init__(self):
        """
        Initializes the BioQAModel.
        """
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
        self.validation_step_outputs = []

    def forward(self, input_ids, attention_mask, labels=None):
        """
        Forward pass of the model.

        Args:
            input_ids (tensor): Input tensor representing tokenized inputs.
            attention_mask (tensor): Tensor representing attention mask.
            labels (tensor, optional): Tensor representing labels. Defaults to None.

        Returns:
            tuple: Tuple containing loss and logits.
        """
        output = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        """
        Training step for the model.

        Args:
            batch (dict): Batch of training data.
            batch_idx (int): Index of the batch.

        Returns:
            tensor: Loss tensor.
        """
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        """
        Validation step for the model.

        Args:
            batch (dict): Batch of validation data.
            batch_idx (int): Index of the batch.

        Returns:
            tensor: Loss tensor.
        """
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.validation_step_outputs.append(loss)
        return loss

    def test_step(self, batch, batch_idx):
        """
        Test step for the model.

        Args:
            batch (dict): Batch of test data.
            batch_idx (int): Index of the batch.

        Returns:
            tensor: Loss tensor.
        """
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        """
        Configure optimizer for the model.

        Returns:
            torch.optim.Optimizer: Optimizer.
        """
        return AdamW(self.parameters(), lr=0.0001)

    def on_validation_epoch_end(self):
        """
        Performs actions at the end of each validation epoch.
        """
        epoch_average = torch.stack(self.validation_step_outputs).mean()
        self.log("validation_epoch_average", epoch_average)
        self.validation_step_outputs.clear()  # free memory

In [ ]:
model = BioQAModel()

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger


# Assuming you have X_train, y_train, X_val, y_val defined

# Define early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min',  # 'min' means training will stop when the quantity monitored has stopped decreasing
)


checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)
logger = TensorBoardLogger("training-logs", name = "bio-qa")
trainer = Trainer(
    callbacks=[checkpoint_callback, early_stopping],
    max_epochs=N_EPOCHS,
    accelerator="gpu",  # Use "gpu" for single GPU, "ddp" for distributed training
    logger=logger
    )

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir ./training-logs

In [ ]:
df_extracted.shape

(100, 4)

In [ ]:
trainer.fit(model,data_module)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /content/checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (12) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 56, global step 684: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 57, global step 696: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 58, global step 708: 'val_loss' was not in top 1
